In [ ]:
import requests
import json

#NIST API URL that only outputs CVEs that are in CISA's Known Exploited Vulnerabilities catalog
# url = "https://services.nvd.nist.gov/rest/json/cves/2.0?hasKev&startIndex=0&cvssV3Severity=CRITICAL&keywordSearch=Microsoft"
url = "https://services.nvd.nist.gov/rest/json/cves/2.0"

### Helper Functions

In [ ]:
def get_nvd_data(url, has_kev=False, start_index=0, cvss_v3_severity=None, keyword_search=None, keyword_exact_match=None):
    #function that receives data from NIST API

    try:
        # conditionally add url params
        params = {"startIndex" : str(start_index)}
        
        if has_kev is not None:
            params["hasKev"] = ""
        
        if cvss_v3_severity is not None:
            params["cvssV3Severity"] = cvss_v3_severity
        
        if keyword_search is not None:
            params["keywordSearch"] = keyword_search
        
        if keyword_exact_match == True:
            params["keywordExactMatch"] = ""

        # not get the results
        response = requests.get(url, params)

        if response.status_code == 200:
            data = response.json()
            return data

        else:
            print(f"Request failed with status code {response.status_code}")
            return None

    except requests.exceptions.RequestException as e:
        print(f"Request failed: {str(e)}")
        return None

In [ ]:
def filter_by_severity(cve_data, filter_severity):
    filtered_results = []

    for vulnerability in cve_data.get("vulnerabilities", []):
        cvss_metric_v2 = vulnerability.get("cve", {}).get("metrics", {}).get("cvssMetricV2", [])
    
        for metric in cvss_metric_v2:
            baseSeverity = metric.get("baseSeverity")

            if baseSeverity.lower() == filter_severity.lower():
                filtered_results.append(vulnerability)

    return filtered_results


In [ ]:
def filter_by_vulnerability_name(cve_data, filter_keyword):
    filtered_results = []

    for vulnerability in cve_data.get("vulnerabilities", []):
        vulnerability_name = vulnerability.get("cve", {}).get("cisaVulnerabilityName", "")
        
        if filter_keyword.lower() in vulnerability_name.lower():
            filtered_results.append(vulnerability)

    return filtered_results

### Main(s)

Online search

In [ ]:
# set params
has_kev = False
start_index = 0
cvss_v3_severity = None # LOW MEDIUM HIGH CRITICAL
keyword_search = "libwebp"
keyword_exact_match = True

# get cve data
cve_data = get_nvd_data(url, has_kev=has_kev, start_index=start_index, cvss_v3_severity=cvss_v3_severity, keyword_search=keyword_search, keyword_exact_match=keyword_exact_match)

In [ ]:
print(f"resultsPerPage: {cve_data['resultsPerPage']}")
print(f"startIndex: {cve_data['startIndex']}")
print(f"totalResults: {cve_data['totalResults']}")

In [ ]:
cve_data

In [ ]:
def extract_version_numbers(cve_data: str):
    versions = []
    if isinstance(cve_data, dict):
        if "versionStartIncluding" in cve_data:
            versions.append(cve_data["versionStartIncluding"])
        if "versionEndExcluding" in cve_data:
            versions.append(cve_data["versionEndExcluding"])
        for key, value in cve_data.items():
            versions.extend(extract_version_numbers(value))
    elif isinstance(cve_data, list):
        for item in cve_data:
            versions.extend(extract_version_numbers(item))
    return versions

In [ ]:
# normalize version to 4 digits each number section
def normalize_version(version, min_dig=4):
    # Split the version string by dots
    parts = version.split('.')

    # Normalize each part to have at least four digits
    normalized_parts = []
    for part in parts:
        normalized_part = part.rjust(min_dig, '0')  # Pad with zeros on the left
        normalized_parts.append(normalized_part)

    # if there are less than four parts, add another section to make all version strings at least 4 number parts
    if len(normalized_parts) < 4:
        normalized_parts.append("0000")

    # Join the normalized parts with dots to form the normalized version
    normalized_version = '.'.join(normalized_parts)

    return normalized_version

In [ ]:
# Function to check if a version is within a range
def is_version_within_range(version, start, end):
    # normalize each version to make it comparable
    version_n = normalize_version(version)

    if start and end:
        start_n = normalize_version(start)
        end_n = normalize_version(end)
        return start_n <= version_n < end_n
    elif start:
        start_n = normalize_version(start)
        return start_n <= version_n
    elif end:
        end_n = normalize_version(end)
        return version_n < end_n
    else:
        return False

In [ ]:
# Function to check if the given version is within any version range
def is_given_version_within_ranges(node, given_version):
    if isinstance(node, dict):
        if "versionStartIncluding" in node or "versionEndExcluding" in node:
            start = node["versionStartIncluding"] if "versionStartIncluding" in node else None
            end = node["versionEndExcluding"] if "versionEndExcluding" in node else None
            if is_version_within_range(given_version, start, end):
                return True
        for key, value in node.items():
            if is_given_version_within_ranges(value, given_version):
                return True
    elif isinstance(node, list):
        for item in node:
            if is_given_version_within_ranges(item, given_version):
                return True
    return False

In [ ]:
version_nums = extract_version_numbers(cve_data)
version_nums

In [ ]:
for vulnerability in cve_data["vulnerabilities"]:
    is_potentially_vulnerable = is_given_version_within_ranges(cve_data, '1.3.4')
    print(f"is_potentially_vulnerable: {is_potentially_vulnerable}    cve: {vulnerability['cve']['id']}")

Filter Local for Severity

In [ ]:
filtered_cve_data = filter_by_severity(cve_data, "HIGH")

In [ ]:
print(f"totalResults: {len(filtered_cve_data)}")

Filter Local for keyword

In [ ]:
filtered_cve_data = filter_by_vulnerability_name(cve_data, "Microsoft")

In [ ]:
len(filtered_cve_data)